In [ ]:
import os
import numpy as np
import MilneEddington as ME
import crisp
import time
import warnings
# Suppress the specific warning
warnings.filterwarnings("ignore", message="The value of the smallest subnormal for <class 'numpy.float32'> type is zero")
warnings.filterwarnings("ignore", message="The value of the smallest subnormal for <class 'numpy.float64'> type is zero")
import inv_utils as iu
from helita.io import lp

from hmi_plot import plot_hmi_ic_mag, plot_sst_pointings

In [ ]:
import importlib
importlib.reload(iu)
print('reloaded') 

In [ ]:
# Load the configuration from the JSON file
input_config = iu.load_yaml_config('input_config.yaml')

In [ ]:
# Check the input configuration
config = iu.check_input_config(input_config, pprint=True, confirm=False)

In [ ]:

# Extract the input parameters
data_dir = config['data_dir']
save_dir = config['save_dir']
crisp_im = config['crisp_im']
xorg = config['xorg']
xsize = config['xsize']
yorg = config['yorg']
ysize = config['ysize']
xrange = config['xrange']
yrange = config['yrange']
tt = config['time_index']
scale = config['scale']
is_north_up = config['is_north_up']
crop = config['crop']
shape = config['shape']
best_frame = config['best_frame']
contrasts = config['contrasts']
hmi_con_series = config['hmi_con_series']
hmi_mag_series = config['hmi_mag_series']
email = config['email']
fov_angle = config['fov_angle']
plot_sst_pointings_flag = config['plot_sst_pointings_flag']
plot_hmi_ic_mag_flag = config['plot_hmi_ic_mag_flag']
plot_crisp_image_flag = config['plot_crisp_image_flag']

In [ ]:
# Extract the fits information from the header
fits_info = config['fits_info']
nx = fits_info['nx']
ny = fits_info['ny']
mu = fits_info['mu']
x1 = fits_info['hplnt'][tt][0]
x2 = fits_info['hplnt'][tt][1]
y1 = fits_info['hpltt'][tt][0]
y2 = fits_info['hpltt'][tt][1]
tobs = fits_info['all_start_times'][tt]
tstart = fits_info['start_time_obs']
tend = fits_info['end_time_obs']
hplnt = fits_info['hplnt']
hpltt = fits_info['hpltt']

In [ ]:
# Reset the x and y ranges if cropping is enabled
if crop:
    x_list = np.linspace(x1, x2, num=nx)
    y_list = np.linspace(y1, y2, num=ny)
    x_list = x_list[xrange[0]:xrange[1]]
    y_list = y_list[yrange[0]:yrange[1]]
    x1 = x_list[0]
    x2 = x_list[-1]
    y1 = y_list[0]
    y2 = y_list[-1]
    nx = xsize
    ny = ysize

In [ ]:
if plot_sst_pointings_flag:
    plot_sst_pointings(tstart, hmi_con_series, hplnt, hpltt,figsize=(6, 6), email=email, save_dir=save_dir)

In [ ]:
if plot_hmi_ic_mag_flag:
    plot_hmi_ic_mag(tobs, hmi_con_series, hmi_mag_series, email, x1, x2, y1, y2, save_dir=save_dir, figsize=(10, 5),  is_north_up=is_north_up, fov_angle=fov_angle, shape=shape)

In [ ]:
if plot_crisp_image_flag:
    print('SST CRISP image with North up:', not(is_north_up))
    iu.plot_crisp_image(crisp_im, tt=tt, ss=0, ww=0, figsize=(6,6), fontsize=10, rot_fov=fov_angle, north_up=not(is_north_up), crop=crop, xrange=xrange, yrange=yrange, xtick_range=[x1,x2], ytick_range=[y1,y2])

In [ ]:
inversion_config = iu.load_yaml_config('inversion_config.yaml')
# Load the variables from the inversion configuration
dtype = inversion_config['dtype']
nthreads = inversion_config['nthreads']
sigma_strength= inversion_config['sigma_strength']
sigma_list = inversion_config['sigma_list']
erh = inversion_config['erh']
init_model = inversion_config['init_model']
nRandom1 = inversion_config['nRandom1']
nIter1 = inversion_config['nIter1']
chi2_thres1 = inversion_config['chi2_thres1']
median_filter_chi2_mean_thres = inversion_config['median_filter_chi2_mean_thres']
median_filter_size = inversion_config['median_filter_size']
nRandom2 = inversion_config['nRandom2']
nIter2 = inversion_config['nIter2']
chi2_thres2 = inversion_config['chi2_thres2']
nIter3 = inversion_config['nIter3']
chi2_thres3 = inversion_config['chi2_thres3']
alpha_strength = inversion_config['alpha_strength']
alpha_list = inversion_config['alpha_list']
nan_mask_replacements = inversion_config['nan_mask_replacements']
show_intermediate_plots = inversion_config['show_intermediate_plots']

In [ ]:
# Load data, wavelength array and cmap
l = iu.container()
iu.container.iwav = iu.get_wavelengths(crisp_im)
iu.container.d = iu.load_crisp_fits(crisp_im, tt=tt, crop=crop, xrange=xrange, yrange=yrange) 
iu.container.cmap = iu.load_crisp_cmap(crisp_im, tt=tt, crop=crop, xrange=xrange, yrange=yrange) 
mask = iu.get_nan_mask(crisp_im, tt=tt,crop=crop, xrange=xrange, yrange=yrange) 

In [ ]:

# Minimum step:
dw = np.min(np.diff(l.iwav))
dw = round(dw*1000.)/1000. # avoid floating point errors
# ==============================================================================
# The inversions need to account for the instrumental
# profile, which involve convolutions. The convolutions
# must be done in a wavelength grid that is at least
# 1/2 of the FWHM of the instrumental profile. In the
# case of CRISP that would be ~55 mA / 2 = ~27.5 mA
#
# Get finer grid for convolutions purposes
# Since we only observed at the lines, let's create
# two regions, one for each line
#
# The observed line positions are not equidistant, the
# Fe I 6301 points only fit into a regular grid of 5 mA
# whereas the Fe I 6302 can fit into a 15 mA grid
#
iw, idx = iu.find_grid(l.iwav, dw)  # Fe I 6173
# ==============================================================================
#
# Now we need to create a data cube with the fine grid
# dimensions. All observed points will contribute to the
# inversion. The non-observed ones will have zero weight
# but will be used internally to properly perform the
# convolution of the synthetic spectra
#
ny, nx = l.d.shape[0:2]
obs = np.zeros((ny, nx, 4, iw.size), dtype=dtype, order='c')
for ss in range(4):
    for ii in range(idx.size):
        obs[:, :, ss, idx[ii]] = l.d[:, :, ss, ii]
# ==============================================================================
#
# Create sigma array with the estimate of the noise for
# each Stokes parameter at all wavelengths. The extra
# non-observed points will have a very large noise (1.e34)
# (zero weight) compared to the observed ones (3.e-3)
# Since the amplitudes of Stokes Q,U and V are very small
# they have a low imprint in Chi2. We can artificially
# give them more weight by lowering the noise estimate.
#
sig = np.zeros((4, iw.size), dtype=dtype) + 1.e32
sig[:, idx] = sigma_strength
sig[1, idx] /= sigma_list[1]
sig[2, idx] /= sigma_list[2]
sig[3, idx] /= sigma_list[3]

# ==============================================================================
#
# Init Me class. We need to create two regions with the
# wavelength arrays defined above and a instrumental profile
# for each region in with the same wavelength step
#
tw = (np.arange(iw.size, dtype=dtype)-iw.size//2)*dw

# ==============================================================================
# Central wavelength of the line:
l0 = iw[iw.size//2]
tr = crisp.crisp(l0).dual_fpi(tw, erh=erh)

regions = [[iw, tr/tr.sum()]]
lines = [int(l0)]
me = ME.MilneEddington(regions, lines, nthreads=nthreads, precision=dtype)

In [ ]:

# Init model parameters
if dtype == np.float32:
    iPar = np.float32(init_model) # [B_tot, theta_B, chi_B, gamma_B, v_los, eta_0, Doppler width, damping, s0, s1]
else:
    iPar = np.float64(init_model) # [B_tot, theta_B, chi_B, gamma_B, v_los, eta_0, Doppler width, damping, s0, s1]
Imodel = me.repeat_model(iPar, ny, nx)


In [ ]:
#
# Run a first cycle with 4 inversions of each pixel (1 + 3 randomizations) of simple pixel-wise inversion
#
t0 = time.time()
Imodel, syn, chi2 = me.invert(Imodel, obs, sig, nRandom=nRandom1, nIter=nIter1, chi2_thres=chi2_thres1, mu=mu)
t1 = time.time()
print("dT = {0}s -> <Chi2> = {1}".format(t1-t0, chi2.mean()))
if show_intermediate_plots:
    iu.plot_inversion_output(Imodel,mask,scale=scale, save_fig=False)
    iu.plot_mag(Imodel,mask,scale=scale, save_fig=False)


In [ ]:
masked_chi2_mean = iu.masked_mean(chi2, mask)
print(f'Masked chi2 mean: {masked_chi2_mean}')
if  masked_chi2_mean < median_filter_chi2_mean_thres[0]:
    size_filter = median_filter_size[0]
elif masked_chi2_mean < median_filter_chi2_mean_thres[1]:
    size_filter = median_filter_size[1]
else:
    size_filter = median_filter_size[2]
Imodel = iu.parallel_median_filter(Imodel, size_filter=size_filter)
if show_intermediate_plots:    
    iu.plot_inversion_output(Imodel,mask,scale=scale, save_fig=False)
    iu.plot_mag(Imodel,mask,scale=scale, save_fig=False)

In [ ]:
#
# Run second cycle
#
t0 = time.time()
Imodel, syn, chi2 = me.invert(Imodel, obs, sig, nRandom=nRandom2, nIter=nIter2, chi2_thres=chi2_thres2, mu=mu)
t1 = time.time()
print("dT = {0}s -> <Chi2> = {1}".format(t1-t0, chi2.mean()))
if show_intermediate_plots:
    iu.plot_inversion_output(Imodel,mask,scale=scale, save_fig=False)
    iu.plot_mag(Imodel,mask,scale=scale, save_fig=False)

In [ ]:

#
# Run a first cycle with 4 inversions of each pixel (1 + 3 randomizations)
#
t0 = time.time()
if dtype == np.float32:
    alphas = np.float32(alpha_list)
else:
    alphas = np.float64(alpha_list)
mo, syn, chi2 = me.invert_spatially_regularized(Imodel, obs, sig,  nIter=nIter3, chi2_thres=chi2_thres3, mu=mu, alpha=alpha_strength, alphas=alphas, method=1, delay_bracket=3)
t1 = time.time()
print("dT = {0}s -> <Chi2> (including regularization) = {1}".format(t1-t0, chi2))

In [ ]:

#
# Correct velocities for cavity error map from CRISP
#
mos = np.squeeze(mo) # Remove the singleton dimension in the model and make the shape (ny, nx, 9) from (1, ny, nx, 9)
mos[:,:,3] += (l.cmap * 10) / l0 * 2.9e5
# mos[:,:,3] += l.cmap+0.45 # The 0.45 is a global offset that seems to make the umbra at rest

In [ ]:
iu.plot_inversion_output(mos,mask,scale=scale, save_fig=False)
iu.plot_mag(mos,mask,scale=scale, save_fig=False)

In [ ]:
iu.plot_sst_blos_bhor(blos_cube, bhor_cube, tt=tt,xrange=xrange, yrange=yrange, figsize=(20,10), fontsize=12, crop=crop)

In [ ]:
errors = me.estimate_uncertainties(mos, obs, sig, mu=mu)

In [ ]:
importlib.reload(iu)

In [ ]:
from einops import rearrange
mos_im = rearrange(mos, 'ny nx nparams -> nparams ny nx')
errors_im = rearrange(errors, 'ny nx nparams -> nparams ny nx')

In [ ]:
for i in range(9):
    iu.masked_stats(mos[:,:,i], mask)

In [ ]:
inversion_mask_replacements = [0, 0, 0, 0, 0, 0, 0, 0, 0] # Blos, inc, azi, v_los, v_dop, line op, damping, s0, s1

In [ ]:
masked_mos = np.zeros_like(mos)
for i in range(9):
    masked_mos[:,:,i] = iu.masked_data(mos[:,:,i], mask, replace_val=inversion_mask_replacements[i])


In [ ]:
iu.plot_inversion_output(masked_mos,scale=scale, save_fig=False)

In [ ]:
masked_errors = np.zeros_like(errors)
for i in range(9):
    masked_errors[:,:,i] = iu.masked_data(errors[:,:,i], mask, replace_val=inversion_mask_replacements[i], fix_inf=True)
iu.plot_inversion_output(masked_errors,scale=scale, save_fig=False)

In [ ]:
for i in range(9):
    iu.masked_stats(errors[:,:,i], mask)

In [ ]:
b_err = iu.masked_data(errors[:,:,0], mask)
print(np.nanmean(b_err))
print(np.nanmin(b_err))
print(np.nanmax(b_err))

In [ ]:
importlib.reload(iu)

In [ ]:
importlib.reload(iu)
iu.plot_image(masked_errors[:,:,1], scale=scale, title='B_tot (G)', save_fig=False, clip=True, vmax=1, vmin=0)

In [ ]:
# apply masked_data to all components or errors and save as masked_errors
masked_errors = np.zeros((ny, nx, 9), dtype=dtype)
for i in range(9):
    masked_errors[:,:,i] = iu.masked_data(errors[:,:,i], mask)

In [ ]:
importlib.reload(iu)
minc = iu.masked_data(errors[:,:,1], mask, replace_val=0)
print(np.min(minc))
print(np.max(minc))
print(np.median(minc))

In [ ]:
iu.plot_inversion_output(masked_errors, mask, scale=scale, save_fig=False)

In [ ]:
iu.plot_output(mos,mask,scale=scale)
iu.plot_mag(Imodel,mask,scale=scale, save_fig=False)

In [ ]:
## save the results as fits files with the same header as the input data
iu.save_fits(mos, fits_header, 'temp/inv_mos.fits', overwrite=True)

In [ ]:
ff = iu.load_fits_data('temp/inv_mos.fits')

In [ ]:
hh = iu.load_fits_header('temp/inv_mos.fits')

#### Things to complete
- [x] Move all the inputs to a dictionary and later save them in the header of the output file. Also add the best seeing frame number.
- [x] Move the preprocessing steps like plotting and FOV details as an optional but default true step
- [x] Plot a rectangle to show cropping region is true
- [ ] Save fits with [blos, theta, phi, vlos + errors + mask] for each frame (temporarily) and later combine for final fits
- [ ] Check for option to convert to fcube and icube formats using ispy or helita tools
- [ ] Add option to do only one frame separately if user wants.
- [ ] Add fov angle and other inputs needed for ambiguity resolution and remap in header

#### To do for final cube
- [x] Pick the best seeing frame from the dataset
- [ ] Run the full inversion for the best seeing frame
- [ ] Use this output as an initial guess for the other frames
